# Experiment 1

In [ ]:
ADD_TENSORBOARD = False

In [ ]:
from src.nb_imports import *

In [ ]:
figures = {}  # collection of ax or figures

In [ ]:
import logging
from src.logging import setup_logger

logger = setup_logger(logger=logging.getLogger('vaep'))
logger.info("Experiment 01")

## Load data

- 1000 features (most abundant peptides)
- later a subset of samples is selected

In [ ]:
N_SAMPLES_TO_LOAD = None
FN_PEPTIDE_INTENSITIES = config.FOLDER_DATA / 'df_intensities_N07813_M01000'
FN_PEPTIDE_INTENSITIES = config.FOLDER_DATA / 'df_intensities_N_00090_M01000'

analysis = AnalyzePeptides(
    fname=FN_PEPTIDE_INTENSITIES, nrows=N_SAMPLES_TO_LOAD)
analysis.df = analysis.df.sort_index()  # sort by date
assert analysis.df.index.is_unique, "Non-unique training samples"
analysis

### Select consecutives samples for training

In [ ]:
from numpy import log2
import random
from vaep.utils import sample_iterable

N_SAMPLES = min(len(analysis.df), 1000)
logger.info(f"Selected {N_SAMPLES}")
analysis.N_SAMPLES = N_SAMPLES

M = 10

columns_selected = sorted(sample_iterable(list(analysis.df.columns), n=M))
analysis.df = analysis.df.loc[:, columns_selected]


def get_consecutive_data_indices(index, n_samples=N_SAMPLES):
    start_sample = len(index) - n_samples
    start_sample = random.randint(0, start_sample)
    return index[start_sample:start_sample+n_samples]


indices_selected = get_consecutive_data_indices(analysis.df.index)
analysis.samples = indices_selected
analysis.df = analysis.df.loc[indices_selected]

LOG_TRANSFORM = log2  # None
if LOG_TRANSFORM:
    analysis.log_transform(LOG_TRANSFORM)

FRACTION = 0.8


class Indices(SimpleNamespace):
    pass


indices = Indices()
indices.train, indices.valid = indices_selected[:int(
    FRACTION*N_SAMPLES)], indices_selected[int(FRACTION*N_SAMPLES):]
analysis.indices = indices

analysis.df_train = analysis.df.loc[indices.train]
analysis.df_valid = analysis.df.loc[indices.valid]

#rebuild original with multi-index
analysis.df_by_split = pd.concat((analysis.df_train, analysis.df_valid), keys=['train', 'valid'])

analysis.df_by_split.sample(n=10, axis=0).sample(n=5, axis=1).sort_index()

### Training and Validation datasets

In [ ]:
n_samples, n_features = analysis.df.shape
msg = "Total:\nN samples: {:10,d} - N Peptides: {:10,d}\n".format(
    n_samples, n_features)
n_train, n_valid = len(analysis.df_train), len(analysis.df_valid)
msg += "N train set: {:8,d} - N valid set: {:9,d}".format(n_train, n_valid)
print(msg)

In [ ]:
analysis.get_dectection_limit()

### Create meta data from filename

In [ ]:
from src import metadata

data_meta = metadata.get_metadata_from_filenames(indices_selected)
analysis.df_meta = pd.DataFrame.from_dict(
    data_meta, orient='index')
# analysis.df_meta['date'] = pd.to_datetime(analysis.df_meta['date'])
analysis.df_meta

- possibility to group data in time along `(machine, lc)` pairs

In [ ]:
analysis.df_meta.loc[indices.train].describe(datetime_is_numeric=False)

In [ ]:
# This becomes part of analysis
def compare_meta_data_for_splits(meta, indices):

    _indices = vars(indices)
    logger.info('Found vars: {}'.format(', '.join(str(x)
                                                  for x in _indices.keys())))

    for key_split, split in _indices.items():
        print(f"{key_split:8} - split description:")
        display(
            meta.loc[split].describe(datetime_is_numeric=True)
        )

    _meta_features = list(meta.columns)

    for _column in _meta_features:
        display(
            _=pd.DataFrame({
                key_split: meta.loc[split, _column].value_counts(normalize=True) for key_split, split in _indices.items()
            }).sort_index().plot(kind='line', rot=90, figsize=(10, 5), title=f"{_column} value Counts for different splits")
        )


compare_meta_data_for_splits(analysis.df_meta.iloc[:, :2], indices)

### PCA plot of original data

In [ ]:
scaler = StandardScaler().fit(analysis.df)
pca = analyzers.run_pca(df=scaler.transform(analysis.df_by_split, copy=None))
cols = list(pca.columns)

fig, axes = plt.subplots(ncols=2, figsize=(15, 8))

# by split
ax = axes[0]
ax = pca.loc['train'].plot.scatter(
    x=cols[0], y=cols[1], color='blue', label='train', ax=ax)
ax = pca.loc['valid'].plot.scatter(
    x=cols[0], y=cols[1], color='orange', label='valid', ax=ax)

# by dates
ax = axes[1]
ax = analyzers.scatter_plot_w_dates(ax, pca, dates=analysis.df_meta.date)

# add colorbar for dates
ax = analyzers.add_date_colorbar(ax=ax, fig=fig)

figures[('pca', 'original')] = fig

- [x] color sample by date (heatmap?)

### Analysis state so far

In [ ]:
analysis

### Correlation

In [ ]:
analyzers.corr_lower_triangle(analysis.df)

### Results

Helper function and results dictionary

In [ ]:
analysis.results = {}


def describe_abs_diff(y_true: pd.DataFrame, y_pred: pd.DataFrame):
    _abs_diff = y_true - y_pred
    return _abs_diff.abs().describe().to_dict()

## Baseline supervised RF models

- M RandomForest baseline models, each predicting one feature based on the M-1 other features
- get an idea of a possible baseline performance
    - could be used together with imputation of inputs
    - with some effort this could be scaled to predict only missing peptides


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
metrics = {}


peptides = list(analysis.df_train.columns)
metrics = {}
pred_valid = {}

for i in range(M):
    train_columns = list(range(M))
    test_column = i
    train_columns.remove(i)
    train_columns = [peptides[i] for i in train_columns]
    test_column = peptides[test_column]
    logger.debug(
        f"Train columns: {', '.join(train_columns)}\nTest column: {test_column}")
    _df_train, _y_train = analysis.df_train[train_columns], analysis.df_train[test_column]
    _df_valid, _y_valid = analysis.df_valid[train_columns], analysis.df_valid[test_column]
    rf_reg = RandomForestRegressor()
    rf_reg.fit(X=_df_train, y=_y_train)
    # metrics
    _metrics = {}
    _metrics[('MSE', 'train')] = mean_squared_error(
        y_true=_y_train, y_pred=rf_reg.predict(_df_train))
    y_pred_valid = rf_reg.predict(_df_valid)
    _metrics[('MSE', 'valid')] = mean_squared_error(
        y_true=_y_valid, y_pred=y_pred_valid)
    metrics[test_column] = _metrics
    # predictions
    pred_valid[test_column] = y_pred_valid
pd.DataFrame(metrics)

In [ ]:
analysis.pred_rf = pd.DataFrame(pred_valid, index=analysis.df_valid.index)
analysis.pred_rf

Overfits to training data as it should.

In [ ]:
analysis.df_valid

In [ ]:
analysis.results['RF baseline'] = describe_abs_diff(
    y_true=analysis.df_valid, y_pred=analysis.pred_rf)
pd.DataFrame(analysis.results['RF baseline'])

Could a model help in identifying extraordinar differences in samples? Something to focus on?

## DL Setup

In [ ]:
import vaep.model as vaep_model
from vaep.cmd import get_args

BATCH_SIZE, EPOCHS = 8, 30
args = get_args(batch_size=BATCH_SIZE, epochs=EPOCHS,
                no_cuda=True)  # data transfer to GPU seems slow
kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}

# torch.manual_seed(args.seed)
device = torch.device("cuda" if args.cuda else "cpu")
device

print(args, device)

## Simple AE
- should also heavily overfit the training data

In [ ]:
from vaep.transform import ShiftedStandardScaler

args_ae = {}
args_ae['SCALER'] = StandardScaler
args_ae['SCALER'] = ShiftedStandardScaler

# select initial data: transformed vs not log transformed
scaler = args_ae['SCALER'](scale_var=2).fit(analysis.df_train)
# five examples from validation dataset
scaler.transform(analysis.df_train).describe(percentiles=[0.025, 0.975])

In [ ]:
scaler.transform(analysis.df_valid).describe(percentiles=[0.025, 0.975])

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
from vaep.io.datasets import PeptideDatasetInMemoryNoMissings

# ToDo: replace with helper class (see below)
tf_norm = None  # replace with Normalizer

dataset_train = PeptideDatasetInMemoryNoMissings(
    data=scaler.transform(analysis.df_train), transform=tf_norm)
dataset_valid = PeptideDatasetInMemoryNoMissings(
    data=scaler.transform(analysis.df_valid), transform=tf_norm)
dl_train = DataLoader(dataset_train, batch_size=args.batch_size, shuffle=True)
dl_valid = DataLoader(dataset_valid, batch_size=args.batch_size, shuffle=False)

### Without Noise

In [ ]:
model = vaep_model.Autoencoder(n_features=M, n_neurons=int(
    M/2), last_decoder_activation=None, dim_latent=3).double()
criterion = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-2

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
)

# Train standard autoencoder (AE)

train_losses, valid_losses = [], []

# do = nn.Dropout()  # for denoising AE
for epoch in range(args.epochs):
    # ===================train==========================
    for data in dl_train:
        model.train()
        data = data.to(device)
        # noise = do(torch.ones(data.shape)).to(device) # for denoising AE
        # data_corrupted = (data * noise).to(device)    # for denoising AE
        # ===================forward=====================
        output = model(data)
        loss = criterion(output, data)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    # ===================validate========================
    for data in dl_valid:
        model.eval()
        data = data.to(device)
        output = model(data)
        loss = criterion(output, data)
        valid_losses.append(loss.item())

    # ===================log=============================
    print(f'epoch [{epoch + 1:03d}/{args.epochs}], '
          f'train-loss: {np.mean(train_losses[-len(dl_train):]):.4f},'
          f'valid-loss: {np.mean(valid_losses[-len(dl_valid):]):.4f}')

In [ ]:
df_train_losses = vaep_model.process_train_loss({'MSE train': train_losses})

# Plotting is boilerplate code:
_ = df_train_losses.plot(kind='scatter', x='steps', y='MSE train smoothed', figsize=(
    15, 8),  title='Exponential smoothed training loss', ylim=(0, None))
df_train_losses.tail()

In [ ]:
def get_pred(model, dataloader):
    pred = []
    model.eval()
    for data in dataloader:
        data = data.to(device)
        output = model(data)
        pred.append(output.detach().numpy())
    return pred


pred = get_pred(model, dl_valid)
analysis.pred_aa_simple = vaep_model.build_df_from_pred_batches(
    pred, scaler, index=analysis.df_valid.index, columns=analysis.df_valid.columns)
analysis.pred_aa_simple

In [ ]:
display(analysis.df_valid)  # true values

In [ ]:
analysis.results['Simple AE'] = describe_abs_diff(
    y_true=analysis.df_valid, y_pred=analysis.pred_aa_simple)
pd.DataFrame(analysis.results['Simple AE'])

### With noise: Denoising AE

- noise is added during training: some values are set to zero (which is the center for standard normalized intensities)
- noise model could be adapted to reflect the observed noise in the training data - > extrapolation to near future should hold

In [ ]:
model = vaep_model.Autoencoder(n_features=M, n_neurons=int(
    M/2), last_decoder_activation=None, dim_latent=3).double()
criterion = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-3

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
)

# Train denoising autoencoder (AE)

train_losses, valid_losses = [], []

do = torch.nn.Dropout()  # for denoising AE
for epoch in range(args.epochs):
    # ===================train==========================
    for data in dl_train:
        model.train()
        data = data.to(device)
        noise = do(torch.ones(data.shape)).to(device)  # for denoising AE
        data_corrupted = (data * noise).to(device)    # for denoising AE
        # ===================forward=====================
        output = model(data)
        loss = criterion(output, data)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    # ===================validate========================
    for data in dl_valid:
        model.eval()
        data = data.to(device)
        output = model(data)
        loss = criterion(output, data)
        valid_losses.append(loss.item())

    # ===================log=============================
    print(f'epoch [{epoch + 1:03d}/{args.epochs}], '
          f'train-loss: {np.mean(train_losses[-len(dl_train):]):.4f},'
          f'valid-loss: {np.mean(valid_losses[-len(dl_valid):]):.4f}')

In [ ]:
df_train_losses = vaep_model.process_train_loss({'MSE train': train_losses})

# Plotting is boilerplate code:
_ = df_train_losses.plot(kind='scatter', x='steps', y='MSE train smoothed', figsize=(
    15, 8),  title='Exponential smoothed training loss', ylim=(0, None))
df_train_losses.tail()

In [ ]:
pred = get_pred(model, dl_valid)
analysis.pred_aa_denoised = vaep_model.build_df_from_pred_batches(
    pred, scaler, index=analysis.df_valid.index, columns=analysis.df_valid.columns)
analysis.results['denoising AE'] = describe_abs_diff(
    y_true=analysis.df_valid, y_pred=analysis.pred_aa_denoised)
pd.DataFrame(analysis.results['denoising AE'])

## Collaborative Filtering setup

Components
- each sample has an embedding vector and an intercept
- each peptide has an embedding vector and an intercept
- scalar product of embeddings yields predictions



In [ ]:
from fastai.collab import CollabDataLoaders, MSELossFlat, Learner
from types import SimpleNamespace
# data format

analysis.collab = Analysis()
collab = analysis.collab
collab.columns = 'peptide,Sample ID,intensity'.split(',')

In [ ]:
analysis.collab

In [ ]:
collab.df = analysis.df.unstack().reset_index(drop=False).rename(
    columns={'level_0': 'peptide', 0: 'intensity'})
collab.df.head()

In [ ]:
dls = CollabDataLoaders.from_df(
    collab.df, user_name='Sample ID', item_name='peptide', rating_name='intensity', bs=64)
dls.show_batch()

In [ ]:
# dls.classes

In [ ]:
import fastai.torch_core
device = torch.device('cpu')
fastai.torch_core.defaults.device = torch.device('cpu')


collab.model_args = {}
collab.model_args['n_samples'] = len(dls.classes['Sample ID'])
collab.model_args['n_peptides'] = len(dls.classes['peptide'])
collab.model_args['dim_latent_factors'] = 5
collab.model_args['y_range'] = (
    int(collab.df['intensity'].min()), int(collab.df['intensity'].max())+1)

collab.model_args

In [ ]:
model = vaep_model.DotProductBias(**collab.model_args)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.summary()

In [ ]:
learn.fit_one_cycle(args.epochs, 5e-3)

In [ ]:
# this shows it along the mini-batches, no easy customization
learn.recorder.plot_loss()

In [ ]:
# learn.recorder.plot_loss??
import matplotlib.pyplot as plt
from fastcore.foundation import L


def plot_loss(self, skip_start=5, with_valid=True, ax=None):
    if not ax:
        fig, ax = plt.subplots()
    ax.plot(list(range(skip_start, len(self.losses))),
            self.losses[skip_start:], label='train')
    if with_valid:
        idx = (np.array(self.iters) < skip_start).sum()
        ax.plot(self.iters[idx:], L(
            self.values[idx:]).itemgot(1), label='valid')
        ax.legend()


fig, ax = plt.subplots(figsize=(10, 8))
plot_loss(learn.recorder, ax=ax)

In [ ]:
# epoch values
for x in learn.recorder.values:
    print(x)
    break

In [ ]:
# get_preds is overloaded, but hardly documented https://docs.fast.ai/learner.html#Learner.get_preds
encodings, pred, target = learn.get_preds(
    with_input=True)  # per default validation data

The analysis concept changes. Here only the (masked) missing peptides could be assessed - without the having entire samples as validation cohorts. Although there is no need for a complete sample, one needs at least some information of a sample to train the sample embedding, leading to a change in the setup.

 - Collaborative Filtering can be trained on all available data to infer the missing peptides

In [ ]:
pred_df = pd.DataFrame([{'Sample ID': dls.classes['Sample ID'][obs[0]], 'peptide': dls.classes['peptide']
                         [obs[1]], 'intensity': pred_intensity.item()} for obs, pred_intensity in zip(encodings, pred)])
pred_df = pred_df.pivot(index='Sample ID', columns='peptide')
pred_df

PyTorch Model used directly:

In [ ]:
valid_dl = learn.dls.valid
model.to(device)
for X, target in valid_dl:
    print(learn.model(X[:1]))
    break

Switching from DotProduct to FNN based on embeddings as implemented in fastai

In [ ]:
from fastai.collab import collab_learner
from fastai.collab import get_emb_sz
# get_emb_sz??

In [ ]:
get_emb_sz(dls)  # default embedding sizes based on dataloader for NN

In [ ]:
collab.model_args  # from above

In [ ]:
from fastai.collab import collab_learner

learn = collab_learner(
    dls, use_nn=True, y_range=collab.model_args['y_range'], layers=[20, 10])
learn.summary()

In [ ]:
learn.fit_one_cycle(30, 5e-3, wd=0.1)

## VAE

### Transform: Non-log transformed data (Single run)

Scale samples according to training data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

args_vae = {}
args_vae['SCALER'] = MinMaxScaler
# select initial data: transformed vs not log transformed
scaler = args_vae['SCALER']().fit(analysis.df_train)
scaler.transform(analysis.df_valid.iloc[:5])

### Dataloaders

In [ ]:
from vaep.io.datasets import PeptideDatasetInMemoryNoMissings
from vaep.io.dataloaders import DataLoadersCreator

data_loader_creator = DataLoadersCreator(
    df_train=analysis.df_train,
    df_valid=analysis.df_valid,
    scaler=scaler,
    DataSetClass=PeptideDatasetInMemoryNoMissings,
    batch_size=args.batch_size)

dl_train, dl_valid = data_loader_creator.get_dls(shuffle_train=True)

logger.info(
    "N train: {:5,d} \nN valid: {:5,d}".format(
        len(dl_train.dataset), len(dl_valid.dataset))
)

### Model

In [ ]:
from torch.nn import functional as F
from torch.nn import Sigmoid
from vaep.model import VAE

n_neurons = 6
logger.info(f'Latent layer neurons: {n_neurons}')

model = vaep_model.VAE(n_features=n_features,
                       n_neurons=n_neurons,
                       last_decoder_activation=Sigmoid,
                       last_encoder_activation=None,
                       dim_latent=4).double()
model = model.to(device)

logger.info(model)

### Optimizers


In [ ]:
from torch import optim
optimizer = optim.Adam(params=model.parameters(),
                       lr=1e-3)

### Tensorboard

In [ ]:
if ADD_TENSORBOARD:
    tensorboard_model_namer = TensorboardModelNamer(
        prefix_folder='experiment_01')
    writer = tensorboard_model_namer.get_writer(1, [n_neurons], 'scaler')
    logger.info(f"Logging to: {writer.get_logdir()}")

    # data, mask = next(iter(dl_train))
    # writer.add_image(
    #     f'{len(mask)} mask for this batch of samples', mask, dataformats='HW')

    data = next(iter(dl_train))
    writer.add_image(
        f'{len(data)} batch of sampled data (as heatmap)', data, dataformats='HW')

    # ToDo: compiler warning: error or tracer error?
    writer.add_graph(model, input_to_model=data.to(
        device))  # try to add after training?
    writer.flush()

### Training Loop

In [ ]:
from collections import defaultdict
from functools import partial

import torch


def run_experiment(model, dls, writer, args):
    metrics = defaultdict(dict)
    metrics_per_batch = defaultdict(list)
    dl_train, dl_valid = dls
    msg_eval_epoch = "Validation Set - Epoch: {:3d} - loss: {:7.3f} - mse: {:5.3f} - KLD: {:5.3f}"

    def _append_batch_metrics(batch_metrics_epoch, d_metrics=metrics_per_batch, dataset_name='train'):
        """Append single batch metrics to global dictionary."""
        for d in batch_metrics_epoch.values():
            for key, value in d.items():
                d_metrics[(dataset_name, key)].append(d[key])
        return None  # Signal in-place operation

    def _agg_metric_per_epoch(batch_metrics_epoch, epoch, d_metrics=metrics, dataset_name='train'):
        keys = next(iter(batch_metrics_epoch.values())).keys()
        for key in keys:
            d_metrics[(dataset_name, key)][epoch] = np.mean([d[key]
                                                             for d in batch_metrics_epoch.values()])
        return None  # Signal in-place operation

    for epoch in range(1, args.epochs+1):
        _epoch_metrics = vaep_model.train(model=model, train_loader=dl_train,
                                          optimizer=optimizer, device=device)
        n_batches = len(dl_train)

        _append_batch_metrics(_epoch_metrics)

        _agg_metric_per_epoch(_epoch_metrics, epoch)

        _epoch_metrics_valid = vaep_model.evaluate(
            model=model, data_loader=dl_valid, device=device)
        n_batches = len(dl_valid)
        _append_batch_metrics(_epoch_metrics_valid, dataset_name='valid')
        _agg_metric_per_epoch(_epoch_metrics_valid,
                              epoch, dataset_name='valid')

        if writer:
            writer.add_scalar('avg validation loss',
                              _epoch_metric_valid['loss'] / n_batchnes,
                              epoch)

    return dict(metrics), dict(metrics_per_batch)

In [ ]:
metrics, metrics_per_batch = run_experiment(model=model, dls=(
    dl_train, dl_valid), writer=None, args=args)  # decide about format

In [ ]:
df_train_losses = vaep_model.process_train_loss(
    {'training loss': metrics_per_batch[('train', 'loss')]})

# Plotting is boilerplate code:
_ = df_train_losses.plot(kind='scatter', x='steps', y='training loss smoothed', figsize=(
    15, 8),  title='Exponential smoothed training loss', ylim=(0, None))
df_train_losses

In [ ]:
metrics_per_batch.keys()

### One epoch

In [ ]:
logger.setLevel(logging.DEBUG)
batch_metrics_last_epoch = vaep_model.train(model=model, train_loader=dl_train,
                                            optimizer=optimizer, device=device)
pd.DataFrame.from_dict(batch_metrics_last_epoch, orient='index')

Currently: No improvements

#### Performance plots

In [ ]:
metrics = pd.DataFrame(metrics)
_ = metrics.plot(
    figsize=(18, 6), xlim=(1, args.epochs))

In [ ]:
metrics

In [ ]:
selected = [(_split, _metric)
            for _split in ['train', 'valid']
            for _metric in ['loss']
            ]
_ = metrics[selected].plot(
    figsize=(18, 6))

### Predictions validation set

In [ ]:
_epoch_metric_valid, pred = vaep_model.evaluate(
    model=model, data_loader=dl_valid, device=device, return_pred=True)
# raw predictions
pd.DataFrame(np.vstack(pred), index=analysis.df_valid.index,
             columns=analysis.df_valid.columns)

In [ ]:
# integrate label in dataloader
analysis.pred_vae = vaep_model.build_df_from_pred_batches(
    pred, scaler, index=analysis.df_valid.index, columns=analysis.df_valid.columns)
analysis.pred_vae

In [ ]:
analysis.df_valid

Absolute differences between VAE prediction and true values

In [ ]:
analysis.results['VAE'] = describe_abs_diff(
    y_true=analysis.df_valid, y_pred=analysis.pred_vae)
pd.DataFrame(analysis.results['VAE'])

Absolute differences in case of mean prediction using **training** data means

In [ ]:
(analysis.df_valid - analysis.df_train.mean()).abs().describe()

### Mean and logvar

In [ ]:
dl_train, dl_valid = data_loader_creator.get_dls(
    shuffle_train=False)  # to have know the samples

latent_space = defaultdict(list)
model.eval()


def _add_pred_to_d(d, k, pred_fct):
    _recon, _mu, _logvar = pred_fct(x)
    _recon, _mu, _logvar = _recon.detach().numpy(), _mu.detach().numpy(), _logvar.detach().numpy()
    d[(k, "mu")].append(_mu)
    d[(k, "logvar")].append(_logvar)
    d[(k, "recon")].append(_recon)


for x in dl_train:
    key = 'train'
    _add_pred_to_d(d=latent_space, k=key, pred_fct=model)
for x in dl_valid:
    key = 'valid'
    _add_pred_to_d(d=latent_space, k=key, pred_fct=model)

# import importlib; importlib.reload(vaep_model)
for (split, stat), arrays in latent_space.items():
    _index = getattr(analysis.indices, split)
    latent_space[(split, stat)] = vaep_model.build_df_from_pred_batches(
        pred=arrays, index=_index)

In [ ]:
latent_space.keys()

PCA plot of latent means

In [ ]:
# analysis.vae.mu
vae_mu = pd.concat([
    latent_space[('train', 'mu')],
    latent_space[('valid', 'mu')]
], keys=['train', 'valid'])
vae_mu

pca = analyzers.run_pca(vae_mu)

In [ ]:
split = 'train'
ax = pca.loc[split].plot.scatter(
    x=cols[0], y=cols[1], title='First two PCs of mu', color='blue', label=split)
split = 'valid'
ax = pca.loc[split].plot.scatter(
    x=cols[0], y=cols[1], title='First two PCs of encoding(mu)', color='orange', label=split, ax=ax)

In [ ]:
cols = list(pca.columns)

fig, axes = plt.subplots(ncols=2, figsize=(15, 8))

# by split
ax = axes[0]

split = 'train'
ax = pca.loc[split].plot.scatter(
    x=cols[0], y=cols[1], title='First two PCs of mu', color='blue', label=split, ax=ax)
split = 'valid'
ax = pca.loc[split].plot.scatter(
    x=cols[0], y=cols[1], title='First two PCs of encoding(mu)', color='orange', label=split, ax=ax)


def scatter_plot_w_dates(ax, df, dates=None):
    """plot first vs. second column in DataFrame.
    Use dates to color data."""

    cols = df.columns

    if isinstance(dates, str):
        dates = df['dates']

    ax = ax.scatter(
        x=df[cols[0]],
        y=df[cols[1]],
        c=[mdates.date2num(t) for t in pd.to_datetime(dates)
           ] if dates is not None else None
    )
    return ax

import matplotlib.dates as mdates
def add_date_colorbar(ax, fig):
    loc = mdates.AutoDateLocator()
    _ = fig.colorbar(ax, ticks=loc,
                 format=mdates.AutoDateFormatter(loc))
    return ax
# by dates
ax = axes[1]
ax = scatter_plot_w_dates(ax, pca, dates=analysis.df_meta.date)

loc = mdates.AutoDateLocator()
_ = fig.colorbar(ax, ticks=loc,
                 format=mdates.AutoDateFormatter(loc))

figures[('pca', 'vae')] = fig

Compare to original PCA

In [ ]:
figures[('pca', 'original')] # pca on std-normalized data

Distance of samples from mean sample

In [ ]:
def euclidian_distance(df, axis=0):
    X = (df - df.mean(axis=axis))**2
    axis = 1-axis
    X = X.sum(axis=axis)
    X = X**0.5
    return X
    

# latent
dist = {}
dist['vae_mu'] =euclidian_distance(vae_mu)

# reconstructed
vae_recon = pd.concat([
    latent_space[('train', 'recon')],
    latent_space[('valid', 'recon')]
], keys=['train', 'valid'])

dist['vae_recon'] =euclidian_distance(vae_recon)

#non-scaled original
dist['original'] = euclidian_distance(analysis.df_by_split)

#scaled original
scaler = StandardScaler().fit(analysis.df_train)
X = scaler.transform(analysis.df_by_split)
dist['original_normalized'] = euclidian_distance(X)

# can different dimensionality be compared directly?

In [ ]:
import itertools
fig, axes = plt.subplots(nrows=len(dist), figsize=(10,7*len(dist)))
axes = itertools.chain(axes)

for i, (key, _s) in enumerate(dist.items()):
    ax = next(axes)
    _ = _s.sort_values().plot(rot=90, ax=ax, title=key)
_ = fig.tight_layout()

In [ ]:
dist = pd.DataFrame(dist)#.sort_values('original')
dist

In [ ]:
_ = dist.sort_values(by='original').plot(rot=90)

## Tensorboard

- can be run from notebook
- or in a separate process to inspect currently running training loops

In [ ]:
if ADD_TENSORBOARD:
    print("Run to see updates: \n\n\ttensorboard "
          f"--logdir {tensorboard_model_namer.folder.absolute()}")

## Compare metrics on AE and VAE

- Collaborative Filtering currently not comparable as setup differs

In [ ]:
analysis.results.keys()

In [ ]:
print("Choose from list of keys: ",
      ", ".join(
          list(next(iter(next(iter(analysis.results.values())).values())).keys()))
      )
_selected_metric = "50%"  # median
print("Currently selected:", _selected_metric)

In [ ]:
# # Comparison Series
# comparison = {(peptide, model_name): stats[_selected_metric] for model_name, description in analysis.results.items() for peptide, stats in description.items()}
# pd.Series(comparison).sort_index()

In [ ]:
# # Comparison as DataFrame
comparison = {}
for model_name, description in analysis.results.items():
    comparison[model_name] = {peptide: stats[_selected_metric]
                              for peptide, stats in description.items()}

pd.DataFrame(comparison).style.apply(vaep.pandas.highlight_min, axis=1)

## Hyperparameter comparison

- [x] order data by date: consecutive samples from training to validation
- [x] check stratification based on machine and column length between splits
    - Do validation and training data have same proportion of machine types? -> generally no, would need to be added
       - not (all) machines are running continously or are continously checked
- [x] complete meta data reading based on filenames
- [x] compare performance regarding data normalization
    - in original intensity space (non-log-transformed) - > 
- [ ] compare performance regarding several hyperparameters of VAE (layers, activation, etc)
    - plot different losses in one plot as validation data set is the same
- [ ] increase number of samples in training set and create result plot
- [ ] increase the number of peptides (features)
- [ ] mask some values in the validation set missing (Quality Assessment)
- [ ] write main function which trains an entire model (including data transformations)
- [ ] add initial PCA plot with samples. Is there any grouping observable? (plotly express)

Debug
- [x] Check reporting of loss again: average sample loss or average peptide loss?
- [x] take a close look at VAE tutorial of PyTorch (data normalization, etc)
- [x] reduce the features size to fewer samples

VAE
- original inputs between 0 and 1 as decoder outputs are transformed originally using the sigmoid fct
- original model use `tanh` activations
- think about the definition of `MSE` in a mini-batch. Should be peptide wise?
    - VAMB does sum over a sample and then takes the mean of the sum (alternative?)
    - multi-output regression?
- learning requires active masking: Mask inputs which should be learned to be recovered. Feed original, 
  not masked image as target to loss.

- [ ] Run MNIST example with MSE loss. Does it still work?
- [x] Normalize inputs to zero and one, use MNIST VAE. Does it work?
  - yes, it learns better then
- [x] Regress M peptide intensities on 1 other peptide intensity. Does it work? (Reference performance)
    - RF baseline model established
- [x] Build a normal AE without probabilistic bottleneck. Does this work?
    - yes

Refactoring

- [x] get epoch out of train, eval etc


Ideas
  - combine 1000 most abundant peptides as guidance for different sets of low abundant peptides
  - show the difference between original and reconstruction using a cm in an Image? batch-wise?

- Current optimum for comparision is zero

> The comparison where relatively low abundant, but not super low-abundant peptides will be masked, could skew the comparison.

In [ ]:
# writer # new writer
# dls = get_dls(data_in_memory, scaler)
# model = VAE()
# writer =  # new writer for each setup
# metrics = run_experiment(model, dls, writer)
# overview['experiment_name'] = metrics

### Inspect batches of the trained model

In [ ]:
index_valid = analysis.df_valid.index
index_train = analysis.df_train.index
columns_ = analysis.df_train.columns

model.eval()

#### Training batch example

In [ ]:
model.to('cpu')
iter_dl_train = iter(dl_train)
batch = next(iter_dl_train)
batch_mask = None
try:
    batch, batch_mask = batch
    batch_masked = batch * batch_mask
except ValueError:
    batch = batch
batch_recon, mu, logvar = model(batch)

In [ ]:
batch_recon

In [ ]:
_batch_metrics = vaep_model.loss_function(batch_recon, batch, mu, logvar)
_batch_metrics

In [ ]:
if batch_mask:
    # avg per peptide loss -> should be close to zero (ref: std=1)
    _mse = ((batch * batch_mask) - (batch_recon * batch_mask)).pow(2).sum()
else:
    _mse = (batch - batch_recon).pow(2).sum()
_mse

In [ ]:
from torch import nn

loss = nn.MSELoss(reduction='sum')
if batch_mask:
    _mse = loss(input=batch_recon*batch_mask, target=batch * batch_mask)
else:
    _mse = loss(input=batch_recon, target=batch)
_mse

In [ ]:
from torch.nn import functional as F
if batch_mask:
    batch_sse = F.mse_loss(input=batch_recon*batch_mask,
                           target=batch * batch_mask, reduction='sum')
else:
    batch_sse = F.mse_loss(input=batch_recon,
                           target=batch, reduction='sum')
batch_sse

####  Validation batch example

In [ ]:
# validation data loader is not shuffled
N_valid = len(dl_valid.dataset)

model.eval()

iter_dl_valid = iter(dl_valid)

batch = next(iter_dl_valid)
batch_mask = None
try:
    batch, batch_mask = batch
    batch_masked = batch * batch_mask
except ValueError:
    batch = batch

batch_recon, mu, logvar = model(batch)

In [ ]:
batch_recon

In [ ]:
_batch_metrics = vaep_model.loss_function(batch_recon, batch, mu, logvar)
_batch_metrics

In [ ]:
if batch_mask:
    # avg per peptide loss -> should be close to zero (ref: std=1)
    _mse = ((batch * batch_mask) - (batch_recon * batch_mask)).pow(2).sum()
else:
    _mse = (batch - batch_recon).pow(2).sum()
_mse

In [ ]:
from torch import nn

loss = nn.MSELoss(reduction='sum')
if batch_mask:
    _mse = loss(input=batch_recon*batch_mask, target=batch * batch_mask)
else:
    _mse = loss(input=batch_recon, target=batch)
_mse

In [ ]:
from torch.nn import functional as F
if batch_mask:
    batch_sse = F.mse_loss(input=batch_recon*batch_mask,
                           target=batch * batch_mask, reduction='sum')
else:
    batch_sse = F.mse_loss(input=batch_recon,
                           target=batch, reduction='sum')
batch_sse

### Inspect Validation data

- VAMB training epoch normalizes by number of batches, [see](https://github.com/RasmussenLab/vamb/blob/734b741b85296377937de54166b7db274bc7ba9c/vamb/encode.py#L284-L335)

In [ ]:
# validation data loader is not shuffled
iter_dl_valid = iter(dl_valid)

batch = next(iter_dl_valid)
batch_mask = None
try:
    batch, batch_mask = batch
    batch_masked = batch * batch_mask
except ValueError:
    batch = batch

M = batch.shape[-1]
batch_recon, _, _ = model(batch)

data = batch.detach().numpy()
if batch_mask:
    mask = batch_mask.detach().numpy()
pred = batch_recon.detach().numpy()

for batch in iter_dl_valid:
    try:
        # ToDo: Test if this works
        if not type(batch) == torch.Tensor:
            batch, batch_mask = batch
            batch_masked = batch * batch_mask
    except ValueError:
        batch = batch
    batch_recon, _, _ = model(batch)
    data = np.append(data, batch.view([-1, M]), axis=0)

    if batch_mask:
        mask = np.append(mask, batch_mask, axis=0)
    pred = np.append(pred, batch_recon.detach().numpy().reshape(-1, M), axis=0)

expected_shape = analysis.df_valid.shape
assert data.shape == expected_shape
assert pred.shape == expected_shape
if batch_mask:
    assert mask.shape == expected_shape

data = pd.DataFrame(data, index=index_valid,
                    columns=columns_).replace(0.0, np.nan)
pred = pd.DataFrame(pred, index=index_valid, columns=columns_)
mask = pd.DataFrame(mask, index=index_valid,
                    columns=columns_) if batch_mask else None

In [ ]:
pd.DataFrame(
    scaler.inverse_transform(pred),
    index=index_valid,
    columns=columns_
)

In [ ]:
metrics.iloc[-1]  # mse loss get's most weight in combined loss

In [ ]:
metrics.iloc[-1].loc[('valid', 'recon_loss')]

Average prediction error per peptides:

-  std. dev is one, so a prediction

In [ ]:
# check that losses reported match loss calculated form predictions
((pred - data)**2).sum().sum() / data.notna().sum().sum()

In [ ]:
(pred - data).iloc[:10, :5]

In [ ]:
((pred - data).iloc[:10, :5])**2

In [ ]:
(pred - data).notna().sum().sum()

In [ ]:
N, M = data.shape
data.isna().sum().sum() / (N*M)  # only few missings